In [2]:
#import packages
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pymannkendall as mk
import cartopy.crs as ccrs
import time
import matplotlib as mpl
from scipy.stats import linregress

In [3]:
#import functions
from functions import *

# DOY trends

## ERA5

### Hotspells

In [26]:
#Prepare data (ERA5 1950-2021)

heat=xr.open_dataset('era5_tmean_heatdays_1950_2024_regrid.nc')
heat1=heat.sortby(heat.time)
#heat79=cuttimestart(heat1,1979)
heat_midl=cutmidlat(heat1)
heat_midlat=landmask(heat_midl)

heat_2021=heat_midlat.sel(time=slice('1950-01-01','2021-12-31'))
heat_doygp=heat_2021.t2m.groupby('time.dayofyear')
#heat_doygp=heat_noleap.t2m.groupby('time.dayofyear')
heat_doytimes=heat_2021.t2m.groupby('time.year').mean().expand_dims({'doys':np.arange(1,366)}).copy().rename({'year':'time'})
for i in range(len(heat_doygp)):
#     if i!=59:
    curgp=heat_doygp[i+1].copy().assign_coords(time=heat_doytimes[i].time)
    heat_doytimes[i,]=curgp
#heat_doytimes2=heat_doytimes.sel(doys=heat_doytimes.doys!=60)

In [27]:
#smooth
heat_doytimesna=heat_doytimes.rename({'doys':'dayofyear'}).dropna(dim='dayofyear',how='all')
heat_doytimes_sm=clim_smoother(heat_doytimesna,90)
heat_doytimes_ds=heat_doytimes_sm.to_dataset(name='t2m')
heat_doy_timesrech=heat_doytimes_ds.chunk({ 'longitude': 11,'dayofyear':61})

In [28]:
#mann kendall (save to files)
slope2, pval2 = xr.apply_ufunc(
        manntrend,
        heat_doy_timesrech.t2m,  # your variable (time on first axis)
        input_core_dims=[['time']],
        output_core_dims=[[], []],
        vectorize=True,
        dask='parallelized',
        output_dtypes=[float, float]
    )

ss2=slope2.compute()
pp2=pval2.compute()
ss2.to_dataset(name='t2m').to_netcdf('hw_tmn_doy_mktrends_smoothed_1950-2021_regrid2.nc')
pp2.to_dataset(name='t2m').to_netcdf('hw_tmn_doy_mkpvals_smoothed_1950-2021_regrid2.nc')

### Temperature 

In [26]:
#get temp data
import dask
era_dmean=xr.open_dataset('era5_tmean_daily_1950_2024_regridcon.nc').load()


era_dmean=era_dmean.sortby(era_dmean.time)
era_dmean=era_dmean.rename({'lat':'latitude','lon':'longitude'})
era_dmean=rotlon_180(era_dmean)
temp_midlat=cutmidlat(era_dmean)
temp_midlat=landmask(temp_midlat)
temp_noleap=temp_midlat.convert_calendar('noleap')
#

In [28]:
#Prepare data (1950-2021)
temp_2021=temp_noleap.sel(time=slice('1950-01-01','2021-12-31'))
temp_doygp=temp_2021.t2m.groupby('time.dayofyear')
#heat_doygp=heat_noleap.t2m.groupby('time.dayofyear')
temp_doytimes=temp_2021.t2m.groupby('time.year').mean().expand_dims({'doys':np.arange(1,366)}).copy().rename({'year':'time'})
for i in range(len(temp_doygp)-1):
#     if i!=59:
    curgp=temp_doygp[i+1].copy().assign_coords(time=temp_doytimes[i].time)
    temp_doytimes[i,]=curgp

In [29]:
#smooth
temp_doytimes=temp_doytimes.rename({'doys':'dayofyear'})
temp_doytimes_sm=clim_smoother(temp_doytimes,90)
temp_doytimes_ds=temp_doytimes_sm.to_dataset(name='t2m')
temp_doy_timesrech=temp_doytimes_ds.chunk({'latitude': 26, 'longitude': 60,'dayofyear':61})

In [32]:
#mann kendall
slope, pval = xr.apply_ufunc(
        manntrend,
        temp_doy_timesrech.t2m,  # your variable (time on first axis)
        input_core_dims=[['time']],
        output_core_dims=[[], []],
        vectorize=True,
        dask='parallelized',
        output_dtypes=[float, float]
    )

In [ ]:
#save to files
ss=slope.compute()
pp=pval.compute()
ss.to_dataset(name='t2m').to_netcdf('tmean_doy_mktrends_1950-2021_regrid.nc')
pp.to_dataset(name='t2m').to_netcdf('tmean_doy_mkpvals_1950-2021_regrid.nc')

## Berkley

### Hotspells

In [35]:
#get hw data
import dask

heat=xr.open_dataset('berkley_tmean_heatdays_1950_2021_regrid.nc')
heat1=heat.sortby(heat.time)
#heat79=cuttimestart(heat1,1979)
heat_midl=cutmidlat(heat1)

heat_midlat=landmask(heat_midl)
#heat_noleap=no_leap(heat_midlat)
#

In [36]:
#Prepare data
heat_doygp=heat_midlat.t2m.groupby('time.dayofyear')
#heat_doygp=heat_noleap.t2m.groupby('time.dayofyear')
heat_doytimes=heat_midlat.t2m.groupby('time.year').mean().expand_dims({'doys':np.arange(1,366)}).copy().rename({'year':'time'})
for i in range(len(heat_doygp)):
#     if i!=59:
    curgp=heat_doygp[i+1].copy().assign_coords(time=heat_doytimes[i].time)
    heat_doytimes[i,]=curgp
#heat_doytimes2=heat_doytimes.sel(doys=heat_doytimes.doys!=60)

In [37]:
#smooth
heat_doytimesna=heat_doytimes.rename({'doys':'dayofyear'}).dropna(dim='dayofyear',how='all')
heat_doytimes_sm=clim_smoother(heat_doytimesna,90)
heat_doytimes_ds=heat_doytimes_sm.to_dataset(name='t2m')
heat_doy_timesrech=heat_doytimes_ds.chunk({ 'longitude': 11,'dayofyear':61})

In [38]:
#mann kendall (save to files)
slope2, pval2 = xr.apply_ufunc(
        manntrend,
        heat_doy_timesrech.t2m,  # your variable (time on first axis)
        input_core_dims=[['time']],
        output_core_dims=[[], []],
        vectorize=True,
        dask='parallelized',
        output_dtypes=[float, float]
    )

ss2=slope2.compute()
pp2=pval2.compute()
ss2.to_dataset(name='t2m').to_netcdf('berkley_tmn_heat_doy_mktrends_smoothed_1950-2021_regrid2.nc')
pp2.to_dataset(name='t2m').to_netcdf('berkley_tmn_heat_doy_mkpvals_smoothed_1950-2021_regrid2.nc')

### Temperature

In [49]:
#get temp data
import dask

bk_dmean=xr.open_dataset('berkley_tmean_1950-2021_regridcon.nc').load()

bk_dmean=bk_dmean.sortby(bk_dmean.time)
bk_dmean=bk_dmean.rename({'lat':'latitude','lon':'longitude'})
bk_dmean=rotlon_180(bk_dmean)
temp_midlat=cutmidlat(bk_dmean)
temp_midlat=landmask(temp_midlat)
temp_noleap=temp_midlat.convert_calendar('noleap')
#

In [51]:
# Prepare data
temp_doygp=temp_noleap.temperature.groupby('time.dayofyear')
#heat_doygp=heat_noleap.t2m.groupby('time.dayofyear')
temp_doytimes=temp_noleap.temperature.groupby('time.year').mean().expand_dims({'doys':np.arange(1,366)}).copy().rename({'year':'time'})
for i in range(len(temp_doygp)-1):
#     if i!=59:
    curgp=temp_doygp[i+1].copy().assign_coords(time=temp_doytimes[i].time)
    temp_doytimes[i,]=curgp
#temp_doytimes=temp_doytimes.sel(doys=temp_doytimes.doys!=60)

In [52]:
#smooth 
temp_doytimes=temp_doytimes.rename({'doys':'dayofyear'})
temp_doytimes_sm=clim_smoother(temp_doytimes,90)
temp_doytimes_ds=temp_doytimes_sm.to_dataset(name='t2m')
temp_doy_timesrech=temp_doytimes_ds.chunk({'latitude': 26, 'longitude': 60,'dayofyear':61})

In [53]:
#mann kendall
slope, pval = xr.apply_ufunc(
        manntrend,
        temp_doy_timesrech.t2m,  # your variable (time on first axis)
        input_core_dims=[['time']],
        output_core_dims=[[], []],
        vectorize=True,
        dask='parallelized',
        output_dtypes=[float, float]
    )

In [54]:
#save to files
ss=slope.compute()
pp=pval.compute()
ss.to_dataset(name='t2m').to_netcdf('bk_tmean_doy_mktrends_1950-2021_regrid.nc')
pp.to_dataset(name='t2m').to_netcdf('bk_tmean_doy_mkpvals_1950-2021_regrid.nc')

## CMIP6

### Hotspells

In [ ]:
#models
###'HadGEM3-GC31-MM'
models1=['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2','CMCC-ESM2','CNRM-ESM2-1','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM5-0','KIOST-ESM','MIROC6','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']# temp_datcut={}
models22=['CESM2-WACCM', 'CMCC-CM2-SR5', 'CNRM-CM6-1','INM-CM4-8','IPSL-CM6A-LR', 'MIROC-ES2L','CNRM-CM6-1-HR']
models=models1+models22#['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2', 'CESM2-WACCM','CMCC-ESM2', 'CMCC-CM2-SR5','CNRM-ESM2-1', 'CNRM-CM6-1', 'CNRM-CM6-1-HR','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR','KIOST-ESM','MIROC6', 'MIROC-ES2L','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']
len(models)
#models=['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2','CMCC-ESM2','CNRM-ESM2-1','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM5-0','KIOST-ESM','MIROC6','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']

In [ ]:
#get hw data
import dask
heat_dat={}
toconc=[]
for i,mod in enumerate(models):
    heat=xr.open_dataset('cmip6/heat/'+mod+'_tmean_heatdays_hist_ssp5_1950_2024_regridcon.nc')#.rename({'lat':'latitude','lon':'longitude'})
    heat1=heat.sortby(heat.time)
    heat_midl=cutmidlat(heat1)
    heat_midlat=landmask(heat_midl)
    heat_midlat=heat_midlat.drop_duplicates(dim="time")
    if i >0:
        heat_midlat['time']=heat_dat['CESM2-WACCM'].time.values
    else:
        pass
    heat_dat[mod]=heat_midlat
    toconc.append(heat_midlat)

In [ ]:
#concatenate
heat_mult=xr.concat(toconc,dim='models')
#heat_mult21=heat_mult.sel(time=slice('1950-01-01','2021-12-31'))

In [ ]:
#prepare dat (1950-2021)
htmmm=heat_mult.sel(time=slice('1950-01-01','2021-12-31'))
heat_doygp=htmmm.htdays.groupby('time.dayofyear')
#heat_doygp=heat_noleap.t2m.groupby('time.dayofyear')
heat_doytimes=htmmm.htdays.groupby('time.year').mean().expand_dims({'doys':np.arange(1,366)}).copy().rename({'year':'time'})
for i in range(len(heat_doygp)):
    #if i!=59:
    curgp=heat_doygp[i+1].copy().assign_coords(time=heat_doytimes[i].time)
    heat_doytimes[i,]=curgp
#heat_doytimes=heat_doytimes.sel(doys=heat_doytimes.doys!=59)

In [ ]:
#Smooth
heat_doytimes_ds=heat_doytimes.assign_coords(models=heat_doytimes.models)
heat_doytimes_ds_m=heat_doytimes_ds.copy()

heat_doytimesna=heat_doytimes_ds_m.rename({'doys':'dayofyear'}).dropna(dim='dayofyear',how='all')
heat_doytimes_sm=clim_smoother(heat_doytimesna,90)
heat_doytimes_ds=heat_doytimes_sm.to_dataset(name='t2m')
heat_doy_timesrech=heat_doytimes_ds.chunk({ 'longitude': 11,'dayofyear':61})

In [ ]:
#chunk and slope

tasks = []
trend_dat=xr.full_like(heat_doytimes_ds.t2m.isel(time=0),np.nan)
pv_dat=xr.full_like(heat_doytimes_ds.t2m.isel(time=0),np.nan)
for i in range(1,365):
    print(i)

#     heatdy_seasch=heat_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
    subset = heat_doytimes_ds.isel(dayofyear=i).chunk({'latitude': 13, 'longitude': 12,'time': -1})
    slope, pval = xr.apply_ufunc(
        manntrend,
        subset.t2m,
        input_core_dims=[['time']],
        output_core_dims=[[], []],
        vectorize=True,
        dask='parallelized',
        output_dtypes=[float, float]
    )
    
    ss,pp=dask.compute(slope,pval)
    
    trend_dat[i,]=ss
    pv_dat[i,]=pp



In [ ]:
#save to files
trend_dat.to_netcdf('cmip_newmods_hw_tmn_doy_tmean_mktrnds_1950-2021.nc')
pv_dat.to_netcdf('cmip_newmods_hw_tmn_doy_tmean_mkpvs_1950-2021.nc')

### Temperature

In [ ]:
#read cmip
# temp_mult=xr.open_dataset('cmip6_tas_day_hist_ssp_models_regridcon.nc').load()
models1=['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2','CMCC-ESM2','CNRM-ESM2-1','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM5-0','KIOST-ESM','MIROC6','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']# temp_datcut={}
models22=['CESM2-WACCM', 'CMCC-CM2-SR5', 'CNRM-CM6-1','INM-CM4-8','IPSL-CM6A-LR', 'MIROC-ES2L','CNRM-CM6-1-HR']
models=models1+models22#['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2', 'CESM2-WACCM','CMCC-ESM2', 'CMCC-CM2-SR5','CNRM-ESM2-1', 'CNRM-CM6-1', 'CNRM-CM6-1-HR','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR','KIOST-ESM','MIROC6', 'MIROC-ES2L','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']
len(models)

In [ ]:
# temp_datcut={}
to_con=[]
for i,mod in enumerate(models):
    dat=xr.open_dataset('cmip6/tmean/tmean_'+mod+'_histssp_regridcon.nc')#.rename({'lat':'latitude','lon':'longitude'})
    dat1=dat.convert_calendar('noleap').rename({'lon':'longitude','lat':'latitude'})
    temp_dat=rotlon_180(dat1)
    tmp1=temp_dat.sortby(temp_dat.time)
    tmp_midl=cutmidlat(tmp1)
    tmp_midlat=landmask(tmp_midl)
    tmp_midlat=tmp_midlat.drop_duplicates(dim="time")
    if i>0:
        tmp_midlat['time']=to_con[0].time.values
   # temp_datcut[mod]=temp_dat#.sel(time=(slice('1950-01-01','2024-12-31')))
    to_con.append(tmp_midlat) 

In [ ]:
#concat
temp_mult=xr.concat(to_con,dim='models')
temp_mult=temp_mult.sortby(temp_mult.time)
temp_mult21=temp_mult.sel(time=slice('1950-01-01','2021-12-31'))
temp_lnd=cutmidlat(temp_mult21)
temp_midlat=landmask(temp_lnd)

In [ ]:
#prepare data
#temp_mult21=temp_mult.sel(time=slice('1950-01-01','2021-12-31'))
htmmm=temp_midlat.copy()#mean(dim='models')
temp_doygp=htmmm.tas.groupby('time.dayofyear')
#temp_doygp=temp_noleap.t2m.groupby('time.dayofyear')
temp_doytimes=htmmm.tas.groupby('time.year').mean().expand_dims({'doys':np.arange(1,366)}).copy().rename({'year':'time'})
for i in range(len(temp_doygp)):
    curgp=temp_doygp[i+1].copy().assign_coords(time=temp_doytimes[i].time)
    temp_doytimes[i,]=curgp


In [ ]:
#smooth

temp_doytimes_ds=temp_doytimes.assign_coords(models=temp_doytimes.models)

temp_doytimes_ds_m=temp_doytimes_ds.copy()

temp_doytimesna=temp_doytimes_ds_m.rename({'doys':'dayofyear'}).dropna(dim='dayofyear',how='all')
temp_doytimes_sm=clim_smoother(temp_doytimesna,90)
temp_doytimes_ds=temp_doytimes_sm.to_dataset(name='t2m')
temp_doy_timesrech=temp_doytimes_ds.chunk({ 'longitude': 11,'dayofyear':61})

In [ ]:
#chunk and slope

import dask

tasks = []
trend_dat=xr.full_like(temp_doytimes_ds.t2m.isel(time=0),np.nan)
pv_dat=xr.full_like(temp_doytimes_ds.t2m.isel(time=0),np.nan)
for i in range(1,365):
    print(i)

#     tempdy_seasch=temp_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
    subset = temp_doytimes_ds.isel(dayofyear=i).chunk({'latitude': 13, 'longitude': 12,'time': -1})
    slope, pval = xr.apply_ufunc(
        manntrend,
        subset.t2m,
        input_core_dims=[['time']],
        output_core_dims=[[], []],
        vectorize=True,
        dask='parallelized',
        output_dtypes=[float, float]
    )
    
    ss,pp=dask.compute(slope,pval)
    
    trend_dat[i,]=ss
    pv_dat[i,]=pp



In [ ]:
#save
trend_dat.to_dataset(name='t2m').to_netcdf('cmip_tmn_newmods_doy_tmean_mktrnds_1950-2021.nc')
pv_dat.to_dataset(name='t2m').to_netcdf('cmip_tmn_newmods_doy_tmean_mkpvs_1950-2021.nc')
#.mean(dim=['latitude','longitude'])
#ss1=sss..compute()#.plot()